In [1]:
from qmachine.pulse_generator.pulse_generator import Pulse_builder
import pandas as pd
import numpy as np

In [2]:
def hold_action(channel,time,looped=False):
    return {'action':'hold' , 'channel':channel , 'action_variables':{'time':time},'looped':looped}

def step_action(channel,value,time,looped=False,looped_variable='step_value',loop_index=0):
    return {'action':'step' , 'channel':channel , 'action_variables':{'time':time,'step_value':value},'looped':looped,'looper':looped_variable,'loop_index':loop_index}

def ramp_action(channel,rate,time,looped=False):
    return {'action':'ramp' , 'channel':channel , 'action_variables':{'time':time,'rate':rate},'looped':looped}

def meas_action(channel,type='full',pulse='readout_pulse_0_05',looped=False,buffer_size=16,slices=100,analog_output='out1'):
    return {'action':'meas' , 'channel':channel , 'action_variables':{'type':type , 'pulse':pulse,'buffer_size':buffer_size,'slices':slices,'analog_output':analog_output},'looped':looped}

def ramp_to_zero_action(channel,time=None):
    return {'action':'ramp_to_zero','channel':channel,'action_variables':{'time':time},'looped':False}

In [3]:
dividers = {'ch1': 8.18*1e-3, 'ch2': 8.27*1e-3} #ch1 is BNC36/j5, ch2 is BNC29/j7, the 1e-3 is to account for values listed being in mV not V
df = pd.read_csv('funnel.csv')
df

,Unnamed: 0,name,time,type,ch1,ch2,m11,m21,m12,m22
0,1,a,10.0,ramp,-6.62,-5.42,False,False,False,True
1,2,b,10.0,ramp,"-6.62, -3.58","-5.42, 6.32",False,False,False,True
2,3,c,15.0,ramp,-1.308,1.117,True,False,False,False
3,4,d,0.2,ramp,"0,30.412,250","0,-25.983,250",False,False,False,True
4,5,e,15.0,ramp,-1.308,1.117,True,False,False,False


In [4]:
def make_zero_avg(df,channel):
    other_cols=[i for i in df.columns.values if 'ch' not in i and 'm1' not in i and 'm2' not in i]
    channel_df=df[[channel]+other_cols].copy(deep=True)
    total_offset=0
    for index,row in channel_df.iterrows():
        row[channel]=row[channel].split(',')
        if len(row[channel])==1:
            total_offset+=float(row[channel][0])*float(row['time'])
        if len(row[channel])==2:
            
            total_offset+=(float(row[channel][1])+float(row[channel][0]))/2*float(row['time']) #ramp
        if len(row[channel])==3:
            total_offset+=np.linspace(float(row[channel][0]),float(row[channel][1]),int(row[channel][2]))*float(row['time'])
        # print(total_offset)

    return calc_offset_comp(total_offset)

def calc_offset_comp(offset,correction_len=30):
    return offset/correction_len


zero_avg_ch1=make_zero_avg(df,'ch1')
zero_avg_ch2=make_zero_avg(df,'ch2')

In [5]:
time1=int(1e4/4)
time2=int(1.5e4/4)
time3=int(200/4)
time4=int(3e4//4)

time1_r=time1*4
time2_r=time2*4
time3_r=time3*4

ch1_rate=(-3.58-(-6.62))/1e4*dividers['ch1'] #V/ns
ch2_rate=(6.32-(-5.42))/1e4*dividers['ch2'] #V/ns
print(f'ch1_rate,total_change: {ch1_rate,ch1_rate*1e4}, ch2_rate,total_change: {ch2_rate,ch2_rate*1e4}')


n_loops=300 #default

actions={'steps':{
                '1':
                    {'ch1':step_action('ch1',-6.62*dividers['ch1'],time1),
                     'ch2':step_action('ch2',-5.42*dividers['ch2'],time1)
                     },
                '2':
                    {'ch1':ramp_action('ch1',ch1_rate,time1),
                     'ch2':ramp_action('ch2',ch2_rate,time1)
                    },
                '3':
                    {
                     'ch1':step_action('ch1',(-0.38-(-3.58))*dividers['ch1'],time2),
                     'ch2':step_action('ch2',(0.325-6.32)*dividers['ch2'],time2),
                     'm1':meas_action('m1','full','readout_pulse_0_2',buffer_size=[n_loops])
                    },
                '4':
                    {'ch1':step_action('ch1',np.linspace(0,(30.416-(-0.38))*dividers['ch1'],n_loops),time3,looped=True,loop_index='0'),
                     'ch2':step_action('ch2',np.linspace(0,(-25.978-(0.325))*dividers['ch2'],n_loops),time3,looped=True,loop_index='0')                        
                    },
                '5':
                    {'ch1':step_action('ch1',-np.linspace(0,(30.416-(-0.38))*dividers['ch1'],n_loops),time2,looped=True,loop_index='0'),
                     'ch2':step_action('ch2',-np.linspace(0,(-25.978-(0.325))*dividers['ch2'],n_loops),time2,looped=True,loop_index='0'),
                     'm1':meas_action('m1','full','readout_pulse_0_2',buffer_size=[n_loops])                        
                    },
                '6':
                    {'ch1':step_action('ch1',zero_avg_ch1*dividers['ch1']-(-0.38*dividers['ch1']),time4,looped=True,loop_index='0'),
                     'ch2':step_action('ch2',zero_avg_ch2*dividers['ch2']-(0.325*dividers['ch2']),time4,looped=True,loop_index='0')
                    },
                '7':
                    {
                    'ch1':ramp_to_zero_action('ch1',1),
                    'ch2':ramp_to_zero_action('ch2',1) #these actions nescessary possibly due to too low resolution on the ramp rate, -> ramp is inprecise?
                    },
                '8':
                    {
                    'ch1':hold_action('ch1',time=time1),
                    # 'ch2':hold_action('ch2',time=time1) #these actions nescessary possibly due to too low resolution on the ramp rate, -> ramp is inprecise?
                    }
                },
        'looped':[n_loops],
        'channels':['ch1','ch2','m1']}




one_loop_time=(3*time1+2*time2+time3+time4)
print(f'loop time in us:{one_loop_time*4/1e3}')


import pprint
pprint.pprint(actions)


ch1_rate,total_change: (2.48672e-06, 0.0248672), ch2_rate,total_change: (9.708979999999998e-06, 0.09708979999999999)
loop time in us:90.2
{'channels': ['ch1', 'ch2', 'm1'],
 'looped': [300],
 'steps': {'1': {'ch1': {'action': 'step',
                         'action_variables': {'step_value': -0.0541516,
                                              'time': 2500},
                         'channel': 'ch1',
                         'loop_index': 0,
                         'looped': False,
                         'looper': 'step_value'},
                 'ch2': {'action': 'step',
                         'action_variables': {'step_value': -0.0448234,
                                              'time': 2500},
                         'channel': 'ch2',
                         'loop_index': 0,
                         'looped': False,
                         'looper': 'step_value'}},
           '2': {'ch1': {'action': 'ramp',
                         'action_variables': {'rate': 2.486

In [6]:
from pprint import pprint
pb = Pulse_builder(dividers)
pb.make_dict(df)
pprint(pb.actions_dict)

{'channels': ['ch1', 'ch2', 'm11'],
 'looped': [250],
 'steps': {'1': {'ch1': {'action': 'step',
                         'action_variables': {'step_value': -6.62,
                                              'time': 2500},
                         'channel': 'ch1',
                         'loop_index': '0',
                         'looped': False,
                         'looper': 'step_value'},
                 'ch2': {'action': 'step',
                         'action_variables': {'step_value': -5.42,
                                              'time': 2500},
                         'channel': 'ch2',
                         'loop_index': '0',
                         'looped': False,
                         'looper': 'step_value'}},
           '2': {'ch1': {'action': 'ramp',
                         'action_variables': {'rate': 0.000304, 'time': 2500},
                         'channel': 'ch1',
                         'looped': False},
                 'ch2': {'action': 'r

In [7]:
for key,value1 in pb.actions_dict['steps'].items():
    for key,value in value1.items():
        if value['looped']:
            value['loop_index']=str(value['loop_index'])

In [8]:
# a note on how to retrieve data when it is saved using .save_all() with no buffering on the stream
# all_data = {name : data.fetch_all() for name,data in job.result_handles._all_results.items()}
# all_data = {name : [k[0] for k in data] for name,data in all_data.items()}

In [9]:
value

{'action': 'ramp_to_zero',
 'channel': 'ch2',
 'action_variables': {'time': 0},
 'looped': False}